In [1]:
import numpy as np
import tensorflow as tf
from keras.layers import Dense
import pandas as pd
from keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


In [2]:
df = pd.read_csv('credit_data.csv')

In [3]:
x_data = np.array(df.iloc[:497, 0:6], dtype=np.float32)
y_data = np.array(df.iloc[:497, 6], dtype=np.float32).reshape(-1, 1)
x_pred = np.array(df.iloc[497:, 0:6], dtype=np.float32)
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.1)

In [5]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))\
            .repeat()\
            .batch(12)\
            .make_one_shot_iterator()\
            .get_next()

In [6]:
nInput = 6      # input layer의 neuron 개수
nHidden = 12    # hidden layer의 neuron 개수
nOutput = 1     # output layer의 neuron 개수

# 그래프를 생성한다
x = tf.placeholder(tf.float32, shape=[None, nInput], name='x')
y = tf.placeholder(tf.float32, shape=[None, nOutput], name='y')

# hidden layer의 Weight (Wh)와 Bias (Bh)
H1 = Dense(nHidden, activation='relu')(x)
predY = Dense(nOutput, activation='sigmoid')(H1)
loss = tf.reduce_mean(binary_crossentropy(y, predY))

# 학습
optimizer = tf.train.AdamOptimizer(0.01)
train = optimizer.minimize(loss)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [7]:
# 그래프를 실행한다
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
for i in range(1000):
    bx, by = sess.run(dataset)
    _, cost_ = sess.run([train, loss], feed_dict={x: bx, y: by})
    if i % 100 == 0:
        print("%d: loss = %.4f" % (i, cost_))

0: loss = 0.2085
100: loss = 0.4269
200: loss = 0.4139
300: loss = 0.3125
400: loss = 0.2845
500: loss = 0.4356
600: loss = 0.3096
700: loss = 0.3057
800: loss = 0.5079
900: loss = 0.3141


In [17]:
yHat = sess.run(predY, feed_dict={x: x_test})
accuracy = accuracy_score(y_test, np.round(yHat))
print("정확도 = %.4f" % accuracy)

정확도 = 0.8000


In [11]:
yPred = sess.run(predY, feed_dict={x: x_pred})
print(yPred)

[[0.8504145 ]
 [0.23027574]]
